In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

import collections
import functools
import abc
import random

import numpy as np
import jax
import jax.numpy as jnp
import jax.experimental.optimizers
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sobol_seq
import dict_minimize

import bvbq
# import bvbq.gp
# import bvbq.bvi
# import bvbq.distributions

In [3]:
ndim = 2
def exponential_quartic(x):
    x1,x2 = x[...,0],x[...,1]
    res = -0.1*x1**4 -0.5*(2*x2-x1**2)**2
#     return -0.5*((x1+1)**2 + (x2 - 1)**2)
    return res

In [4]:
x = np.random.rand(5,2)

distrib = bvbq.distributions.DiagonalNormalDistribution(-jnp.ones(2),0.1*jnp.ones(2))
print(distrib.sample(100).mean(axis=0))
print(distrib.logprob(x))
print(distrib.sample(101).mean(axis=0))
mixdistrib = distrib.make_mixture()
mixdistrib.add_component(jnp.ones(2),0.1*jnp.ones(2),0.1)
mixdistrib.add_component(2*jnp.ones(2),0.1*jnp.ones(2),0.1)
print(mixdistrib.logprob(x))
print(mixdistrib.sample(100).mean())
print(mixdistrib.sample(101).mean())

W0806 15:04:25.793346 139889439369024 xla_bridge.py:137] No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


[-1.00437   -1.0531996]
[-25.089006 -15.40317  -20.902403 -32.511047 -21.999987]
[-0.9081288 -1.0328283]
[-5.058856  -7.8872323 -7.092367  -2.303052  -4.6215954]
-0.66964185
-0.60675174


In [20]:
x = mixdistrib.means[0,:]
f = mixdistrib.logprob
df = jax.grad(mixdistrib.logprob)
dfv = lambda x,v : jnp.sum(df(x)*v)
ddfv = jax.grad(dfv,argnums=0)


The slowest run took 45.94 times longer than the fastest. This could mean that an intermediate result is being cached.
85.9 ms ± 55.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
